### 서울 시민 금융 데이터를 활용한 탄소 배출량 및 폐기물 분석 시각화 고도화

> "소비 데이터를 토대로 탄소 + 폐기물 발생량의 특성 파악" <br/>
> 탄소 발자국(炭素-, 영어: carbon footprint)은 개인 또는 단체가 직접·간접적으로 발생시키는 온실 기체의 총량을 의미한다. 여기에는 이들이 일상 생활에서 사용하는 연료, 전기, 용품 등이 모두 포함된다
    
- Carbon Emission from Spending : C.E.P
- 탄소 배출량이 높은 품목을 리스트화 
- 탄소 배출량 및 폐기물 배출량 데이터
- 서울시민의 업종별 카드소비 패턴 데이터와 타 금융의 소비 패턴 데이터


### 사용한 실제 데이터 리스트 링크

1. https://bigdata.seoul.go.kr/data/selectSampleData.do?r_id=P213&sample_data_seq=324&tab_type=&sch_cate=90&file_id=&sch_text=&sch_order=U&currentPage=1

2. https://bigdata.seoul.go.kr/data/selectSampleData.do?r_id=P213&sample_data_seq=318&tab_type=&sch_cate=90&file_id=&sch_text=&sch_order=U&currentPage=1

3. https://netis.kemco.or.kr/

In [40]:
# module install

%pip install numpy
%pip install pandas
%pip install matplotlib


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Using cached matplotlib-3.6.0-cp39-cp39-macosx_10_12_x86_64.whl (7.3 MB)
  Using cached Pillow-9.2.0-cp39-cp39-macosx_10_10_x86_64.whl (3.1 MB)
  Using cached contourpy-1.0.5-cp39-cp39-macosx_10_9_x86_64.whl (241 kB)
  Using cached kiwisolver-1.4.4-cp39-cp39-macosx_10_9_x86_64.whl (65 kB)
  Using cached cycler-0.11.0-py3-none-any.whl (6.4 kB)
  Using cached fonttools-4.37.3-py3-none-any.whl (959 kB)
Note: you may need to restart the kernel to use updated packages.


In [2]:

# 사전 세팅 및 import
# To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" # "all" 이면, jupyter output 여러개, 한 꺼 번에 볼 수 있음 

import math
import numpy as np
import pandas as pd

DATA_PATH: str = "./datas/cep_data"
FILE_LIST = os.listdir(DATA_PATH) # 해당 directory에 file name을 모두 불러와 List로 저장
FILE_LIST

def get_df_from_target_data(file_idx: int):
    try:
        return pd.read_csv(DATA_PATH + "/" + FILE_LIST[file_idx], index_col = False, encoding='utf-8')
    except UnicodeDecodeError:
        return pd.read_csv(DATA_PATH + "/" + FILE_LIST[file_idx], index_col = False, encoding='cp949')


In [3]:

# 행정동과, 생활인구 데이터 사전 세팅해두기
BASE_DATA_LIST = os.listdir("./datas/생활이동_행정동_202206/")
BASE_DATA_LIST.sort()
# BASE_DATA_LIST
행정동코드 = pd.read_csv("./datas/dong_code.csv", index_col = False, encoding='cp949')
# 행정동코드

# 행정동 기준 이동 인구 예시 데이터
# pd.read_csv(f"./datas/생활이동_행정동_202206/생활이동_행정동_2022.06_04시.csv", index_col = False, encoding='cp949')

# 목표 "불금 - 토, 13시에서 23시 젊은 층의 이동만 보고 싶다"
for i, j in enumerate(range(13, 24)):
    target_file = BASE_DATA_LIST[j]
    tmp = pd.read_csv(f"./datas/생활이동_행정동_202206/{target_file}", index_col = False, encoding='cp949')
    idx1 = (tmp['요일'] == '금') | (tmp['요일'] == '토')
    idx2 = (tmp['나이'] >= 20) & (tmp['나이'] <= 40)    # 나이는 2~40, 
    idx = idx1 & idx2
    tmp1 = tmp.loc[idx,:] # 필터링한 인덱스 세팅    

    # 출발 행정동 코드, 도착 행정동 코드, 이동인구(합)
    tmp1 = tmp1.iloc[:,[3,4,9]] # 필요한 컬럼의 index 로 세팅해서, 해당 컬럼만 쓰겠다는거
    idx = tmp1['이동인구(합)']=="*"
    tmp1['이동인구(합)'][idx] = 1.0 # * 인 값이 있는데, 이동 인구 합을 1로 가정하고 가자

    tmp1['이동인구(합)'] = tmp1['이동인구(합)'].astype('float32') # str -> float 타입 캐스팅해주는 것
    idx = tmp1['이동인구(합)'] >= 1 # 무조건 이렇게 되어야 하는데, 실수 했는가 싶어서 체크, "< 1" 조건으로 나오는게 0개여야 함

    tmp1 = tmp1.loc[idx,:]
    tmp1 = tmp1.groupby(['출발 행정동 코드', '도착 행정동 코드'])['이동인구(합)'].sum() # 키와 더할(연산) 값(컬럼) 으로 세팅 후 연산 
    tmp1 = tmp1.reset_index() # 나중에 필요해서 컬럼에 넣어버림

    if i==0: # 최초일때, rdata init 
        rdata = tmp1
        next
    
    # next 이후, rdata init 데이터에서 concat
    rdata = pd.concat([rdata, tmp1], axis = 0)

print("rdata 👇")
rdata.head()

rdata_L = rdata.groupby(['출발 행정동 코드', '도착 행정동 코드'])['이동인구(합)'].sum() # 시간대별로 다시 그루핑
rdata_L = rdata_L.reset_index()
print("rdata_L 👇")
rdata_L

<function next>

rdata 👇


,출발 행정동 코드,도착 행정동 코드,이동인구(합)
0,21000,1101053,53.68
1,21000,1101054,20.43
2,21000,1101055,2.00
3,21000,1101056,1.00
4,21000,1101057,9.50


rdata_L 👇


,출발 행정동 코드,도착 행정동 코드,이동인구(합)
0,21000,1101053,267.200012
1,21000,1101054,78.370003
2,21000,1101055,47.059998
3,21000,1101056,27.549999
4,21000,1101057,36.480000
...,...,...,...
604814,3138041,1125063,4.440000
604815,3138041,1125066,12.500000
604816,3138041,1125067,1.000000
604817,3138041,1125071,4.430000


In [5]:

행정동코드

dong_code1 = set(rdata_L["출발 행정동 코드"].unique())
dong_code2 = set(rdata_L["도착 행정동 코드"].unique())
dong_code = list(set.union(dong_code1, dong_code2))
dong_code.sort() # 동코드 작은 것 부터 번호 붙이려고 소팅
# dong_code


,통계청행정동코드,행자부행정동코드,시도명,시군구명,행정동명
0,H_SDNG_CD,H_DNG_CD,DO_NM,CT_NM,H_DNG_NM
1,1101053,11110530,서울,종로구,사직동
2,1101054,11110540,서울,종로구,삼청동
3,1101055,11110550,서울,종로구,부암동
4,1101056,11110560,서울,종로구,평창동
...,...,...,...,...,...
420,1125070,11740690,서울,강동구,둔촌1동
421,1125071,11740700,서울,강동구,둔촌2동
422,1125072,11740570,서울,강동구,암사1동
423,1125073,11740610,서울,강동구,천호2동


In [6]:

#### 판매자 기준, 분류 merge, 블록별 상품판매 DF
카드소비_업종코드 = get_df_from_target_data(3)
카드소비_업종코드['업종코드(UPJONG_CD)'] = 카드소비_업종코드['업종코드(UPJONG_CD)'].str.upper()
카드소비_업종코드
블록별_시간대별소액결제건수_카드소비패턴 = get_df_from_target_data(2)
# 블록별_시간대별소액결제건수_카드소비패턴

롯데멤버스_상품분류표 = get_df_from_target_data(0)
통계청분류 = pd.DataFrame()
통계청분류['통계청분류코드'] = 롯데멤버스_상품분류표['통계청분류코드']
통계청분류['통계청분류명'] = 롯데멤버스_상품분류표['통계청분류명']
통계청분류 = 통계청분류.drop_duplicates()

판매지별_구매자지역기준_블록별_상품판매 = get_df_from_target_data(5)
판매지별_구매자지역기준_블록별_상품판매 = pd.merge(판매지별_구매자지역기준_블록별_상품판매, 통계청분류, how='left', left_on='통계청상품코드(STAT_CD)', right_on='통계청분류코드')
판매지별_구매자지역기준_블록별_상품판매 = 판매지별_구매자지역기준_블록별_상품판매.drop(['통계청상품코드(STAT_CD)'], axis='columns')
판매지별_구매자지역기준_블록별_상품판매['탄소배출량'] = {}
판매지별_구매자지역기준_블록별_상품판매['통계청분류명'].unique()
판매지별_구매자지역기준_블록별_상품판매

#### 일별, 시간대별 카드 소비 패턴 DF
집계구별_일별시간대별_카드소비패턴 = get_df_from_target_data(-3)
# 집계구별_일별시간대별_카드소비패턴 = pd.merge(집계구별_일별시간대별_카드소비패턴, 카드소비_업종코드, how='left', left_on='서울시민업종코드(UPJONG_CD)', right_on='업종코드(UPJONG_CD)')
집계구별_일별시간대별_카드소비패턴


#### 카드 소비 패턴, 업종 코드 merge, 시간대별결제건수 DF
df = pd.merge(블록별_시간대별소액결제건수_카드소비패턴, 카드소비_업종코드, how='left', left_on='서울시민업종코드(UPJONG_CD)', right_on='업종코드(UPJONG_CD)')
df = df.drop(['업종코드(UPJONG_CD)'], axis='columns')
df


,업종코드(UPJONG_CD),대분류(CLASS1),중분류(CLASS2),소분류(CLASS3)
0,SS001,요식/유흥,한식,한식
1,SS002,요식/유흥,일식/중식/양식,일식
2,SS003,요식/유흥,일식/중식/양식,양식
3,SS004,요식/유흥,일식/중식/양식,중식
4,SS005,요식/유흥,제과/커피/패스트푸드,제과점
...,...,...,...,...
70,SS084,스포츠/문화/레저,스포츠/문화/레저,스키
71,SS090,교육/학원,학원,입시보습학원
72,SS091,교육/학원,학원,외국어학원
73,SS092,교육/학원,학원,예체능학원


/var/folders/wq/szw2crw94zl_n5gdv9c7ksd40000gn/T/ipykernel_8466/1111054745.py:17: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  판매지별_구매자지역기준_블록별_상품판매['탄소배출량'] = {}


array(['오락 및 문화', '가정용품 및 가사 서비스', '보건', '식료품 및 비주류음료', '주류 및 담배',
       '기타 상품 및 서비스', '교육', '의류 및 신발', '교통'], dtype=object)

,기준년월(STD_YM),블록코드(BLCK_CD),성별코드(SEX_CD),연령대코드(AGE_CD),시간대코드(TIME_CD),구매자지역(BUYER_AREA),구매고객수(ACC_CNT),구매건수(PURH_CNT),구매금액(PURH_AMT),통계청분류코드,통계청분류명,탄소배출량
0,201911,206648,2,2,6,11680720,7,12,77000,I,오락 및 문화,NaN
1,201805,225172,1,3,6,11710520,3,1,8000,E,가정용품 및 가사 서비스,NaN
2,201905,224659,1,3,6,11710632,1,10,10000,F,보건,NaN
3,201805,413188,2,4,4,11710580,3,95,23000,A,식료품 및 비주류음료,NaN
4,201803,364511,1,2,4,11680656,6,1,1000,A,식료품 및 비주류음료,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
495,201809,274538,2,3,2,11650621,1,1,1000,I,오락 및 문화,NaN
496,201804,353409,2,3,4,11470510,4,2,139000,A,식료품 및 비주류음료,NaN
497,201806,223026,1,4,5,41190,1,3,40000,A,식료품 및 비주류음료,NaN
498,201907,8661,2,4,5,11650651,3,2,4000,A,식료품 및 비주류음료,NaN


,업종대분류(UPJONG_CLASS1),기준일자(YMD),시간대구간(TIME),고객주소집계구별(TOT_REG_CD),카드이용금액계(AMT_CORR),카드이용건수계(USECT_CORR)
0,요식/유흥,20161004,6,1123076010009,104624,10
1,유통,20210303,5,1122060030003,382592,20
2,유통,20170606,6,1123066022301,341688,5
3,스포츠/문화/레저,20171206,5,1124075020103,925118,5
4,의료,20160909,6,1113075030002,217985,50
...,...,...,...,...,...,...
495,전자상거래,20170727,2,1120069010006,90540,25
496,유통,20190406,3,1123072010303,1211325,20
497,교육/학원,20210107,3,1119072030101,266590,15
498,전자상거래,20200426,4,1105063030203,547264,25


,서울시민업종코드(UPJONG_CD),기준년월(YM),시간대구간(TIME),고객주소블록코드(BLOCK_CD),카드이용금액계(AMT_CORR),소액결제건수(MICRO_PYM),대분류(CLASS1),중분류(CLASS2),소분류(CLASS3)
0,SS017,202006,6,11063,257,25,유통,기타유통,기타유통
1,SS013,201608,6,216137,86,10,유통,할인점/슈퍼마켓,할인점/슈퍼마켓
2,SS016,201802,3,224819,35,15,유통,편의점,편의점
3,SS055,201906,2,19869,10,0,의료,병원,일반병원
4,SS038,201612,5,28382,10,5,여행/교통,여행,항공
...,...,...,...,...,...,...,...,...,...
495,SS017,201911,5,366735,5,5,유통,기타유통,기타유통
496,SS016,201803,2,155750,10,30,유통,편의점,편의점
497,SS081,202103,3,366426,5,5,스포츠/문화/레저,스포츠/문화/레저,실내골프
498,SS069,201707,3,214935,70,10,전자상거래,전자상거래,결제대행(PG)


In [102]:

# "전기", 가스, 수도, 교통

# 전기요금
# if(price > 0 && price <= 1130){
#     imsiKwh = 43;
# }else if( price > 1130 && price <= 17703 ){ 
#     imsiKwh = Math.round((price + 3513.33) / 106.0821);	
# }else if(price > 17703 && price <= 23239){
#     imsiKwh = 200;
# }else if(price > 23239 && price <= 65764 ){
#     imsiKwh = Math.round((price + 19692.84) / 213.6423);
# }else if(price > 65764 && price <= 72559){
#     imsiKwh = 400;
# }else{
#     imsiKwh = Math.round( (price + 55371.9) / 319.0422 )
# }

# elecCo2 =  Math.round(( imsiKwh  * 0.4663 ) / 0.1) * 0.1;
# printNumber('co2' , calc(elecCo2));

# if(elecCo2 == 0){
#     elecTree = 0;
# }else{
#     elecTree = Math.round(  ((imsiKwh * 0.4663) / 6.6) / 0.1) * 0.1;
# }

월전기사용량_kwh = 1000
elecCo2 =  round(( 월전기사용량_kwh  * 0.4663 ) / 0.1) * 0.1 # co2 발생량
elecTree = round( ((월전기사용량_kwh * 0.4663) / 6.6) / 0.1) * 0.1 # 필요한 소나무 개수
elecCo2, elecTree



(466.3, 70.7)


## "분류별 탄소배출량"

### 서울시민업종코드별 x 탄소배출량
- 서울시민업종코드별 대분류 (5 5 4) : [
        '요식/유흥', '유통', '음/식료품', '의류/잡화', '스포츠/문화/레저', 
        '여행/교통', '미용', '가정생활/서비스', '교육/학원', '의료', 
        '가전/가구', '자동차', '주유', '전자상거래'
    ]
-  정현우 / 범수님이 / 기원님이 
- 원 단위 곱 "계수" 가 탄소배출량 형태로, 계산식 -> "각 계수 점화식"
    - "ss001","요식/유흥","한식","한식"
    - "ss002","요식/유흥","일식/중식/양식","일식"
    - "ss003","요식/유흥","일식/중식/양식","양식"
    - "ss004","요식/유흥","일식/중식/양식","중식"
    - "ss005","요식/유흥","제과/커피/패스트푸드","제과점"
    - "ss006","요식/유흥","제과/커피/패스트푸드","커피전문점"
    - "ss007","요식/유흥","제과/커피/패스트푸드","패스트푸드"
    - "ss008","요식/유흥","기타요식","기타요식"
    - "ss009","요식/유흥","유흥","노래방"
    - "ss010","요식/유흥","유흥","기타유흥업소"
    - "ss011","요식/유흥","유흥","유흥주점"
    - "ss012","유통","백화점","백화점"
    - "ss013","유통","할인점/슈퍼마켓","할인점/슈퍼마켓"
    - "ss014","유통","할인점/슈퍼마켓","슈퍼마켓"
    - "ss015","유통","할인점/슈퍼마켓","생활잡화"
    - "ss016","유통","편의점","편의점"
    - "ss017","유통","기타유통","기타유통"
    - "ss018","음/식료품","음/식료품","정육점"
    - "ss019","음/식료품","음/식료품","농수산물"
    - "ss020","음/식료품","음/식료품","기타음/식료품"
    - "ss021","의류/잡화","의복/의류","의복/의류"
    - "ss022","의류/잡화","패션/잡화","패션/잡화"
    - "ss023","의류/잡화","패션/잡화","시계/귀금속"
    - "ss024","의류/잡화","패션/잡화","안경"

        ---
        
    - "ss027","스포츠/문화/레저","스포츠/문화/레저","종합레저시설"
    - "ss028","스포츠/문화/레저","스포츠/문화/레저","영화/공연"
    - "ss029","스포츠/문화/레저","스포츠/문화/레저","스포츠시설"
    - "ss030","스포츠/문화/레저","스포츠/문화/레저","취미/오락"
    - "ss031","스포츠/문화/레저","스포츠/문화/레저","서점"
    - "ss032","스포츠/문화/레저","스포츠/문화/레저용품","스포츠/레저용품"
    - "ss033","스포츠/문화/레저","스포츠/문화/레저용품","문화용품"
    - "ss034","스포츠/문화/레저","스포츠/문화/레저용품","화원"
    - "ss081","스포츠/문화/레저","스포츠/문화/레저","실내골프"
    - "ss082","스포츠/문화/레저","스포츠/문화/레저","헬스"
    - "ss083","스포츠/문화/레저","스포츠/문화/레저","실외골프"
    - "ss084","스포츠/문화/레저","스포츠/문화/레저","스키"
    - "ss035","여행/교통","숙박","호텔/콘도"
    - "ss036","여행/교통","숙박","모텔/여관/기타숙박"
    - "ss037","여행/교통","여행","여행사"
    - "ss038","여행/교통","여행","항공"
    - "ss039","여행/교통","여행","면세점"
    - "ss040","여행/교통","교통","교통"
    - "ss041","미용","미용서비스","미용실"
    - "ss042","미용","미용서비스","미용서비스"
    - "ss043","미용","화장품","화장품"
    - "ss044","가정생활/서비스","서비스","생활서비스"
    - "ss045","가정생활/서비스","서비스","세탁소"
    - "ss046","가정생활/서비스","서비스","업무서비스"
    - "ss047","가정생활/서비스","인테리어","인테리어"
    - "ss048","가정생활/서비스","각종요금","통신"
    - "ss049","가정생활/서비스","각종요금","보험"

        ---

    - "ss050","교육/학원","학원","독서실"
    - "ss052","교육/학원","유아교육","유아교육"
    - "ss053","교육/학원","교육용품","교육용품"
    - "ss090","교육/학원","학원","입시보습학원"
    - "ss091","교육/학원","학원","외국어학원"
    - "ss092","교육/학원","학원","예체능학원"
    - "ss093","교육/학원","학원","취미/전문학원"
    - "ss054","의료","병원","종합병원"
    - "ss055","의료","병원","일반병원"
    - "ss056","의료","병원","치과병원"
    - "ss057","의료","병원","한의원"
    - "ss058","의료","약국","약국"
    - "ss059","의료","기타의료","기타의료"
    - "ss060","가전/가구","가전/가구","가전"
    - "ss061","가전/가구","가전/가구","가구"
    - "ss062","가전/가구","가전/가구","기타가전/가구"
    - "ss063","자동차","자동차판매","자동차판매"
    - "ss064","자동차","자동차서비스/용품","자동차서비스"
    - "ss065","자동차","자동차서비스/용품","자동차용품"
    - "ss066","주유","주유","주유소"
    - "ss067","주유","주유","LPG"
    - "ss068","전자상거래","전자상거래","온라인거래"
    - "ss069","전자상거래","전자상거래","결제대행(PG)"
    - "ss070","전자상거래","전자상거래","홈쇼핑"


### 탄소배출량 -> 계산식은? 찾아봐야한다
- https://www.kcen.kr/tanso/intro.green


## "샘플데이터 기반으로 샘플 만들어서?"
- 만들어서
- 블록별 성별연령대별 카드소비패턴 x 위 분류별 탄소배출량 묶은 DF (1차)
- 카드소비_업종코드

In [17]:

# 탄소 배출량 데이터 만들기
# 원당 배출량 계산?
# "총 액" / "총 배출 량" 비율을 알아야 그나마 근접할 수 있다
# 위 계산이 어렵다면 어떻게 접근해야 할까? 
# - 업종당 평균 배출을 임의로 해버리면?
# - 아니면 가중치, 즉 스코어를 줘 버리는건? -> 업종별 총 배출량 랭킹
#   - 이 경우 업종별로 총 소비를 묶어버림 (그럴 필요가 없는게 업종 별 매출 추이는 알 수 있음, 따로 통계 데이터가 있음)
#   - "얼마"를 썻느냐보다 "어디에 썻느냐"에 집중하는 형태
# ss01 ~ ss024
카드소비_업종코드.loc[카드소비_업종코드["업종코드(UPJONG_CD)"] >= "SS024"]
카드소비_업종코드['탄소배출 가중치'] = np.random.randint(1, 100, 카드소비_업종코드.shape[0])
# tCO2eq
# 음식점업 : 16,760,350.6	
# 주점및비알코올 음료점업 : 2,158,355.6

# ---

# 1) 업종 어디에 무엇을 합쳤는지, 2) 1,000원당 탄소배출량, 3) 고객주소블록코드(BLOCK_CD) -> 행정동
# "분석 단위"는 시간(기준 연월)과 행정동별로 
# ex) 해당 연령대에 비중에 따라 탄소배출 증감율 시각화
# ex) 신한_행정동별_성별연령대별_카드소비패턴

# 14개 정도 / "의류/잡화"  ... 13개 -> 분류에 대한 비중은 카드 지출 액 기준
# "각 분류 카드이용금액계(AMT_CORR) / 묶이는 데이터 카드이용금액계"
# 목표 분석 DF) 기준년월(YM) | 고객주소블록코드(BLOCK_CD) - 행정동 | 성별 F (66%) | 연령대(10대) | 연령대(20대) | 연령대(30대) ... | 의류/잡화, 여행/교통  ... 13개 | 
# 총 탄소배출량 (종속변수, 결과, "덧셈") // => 의류/잡화 800000, 1000 * 계수 + 여행/교통 ... 

# 지도, 행정동 별 시각화 

# 각 feature가 값이 달라 졌을 때, 총 탄소배출량 의 변화량 -> 표 

# 행정동별 소비패턴 및 탄소배출량 등 특징/차이 시각화

# +a) 

,업종코드(UPJONG_CD),대분류(CLASS1),중분류(CLASS2),소분류(CLASS3),탄소배출 가중치
23,SS024,의류/잡화,패션/잡화,안경,27
24,SS027,스포츠/문화/레저,스포츠/문화/레저,종합레저시설,73
25,SS028,스포츠/문화/레저,스포츠/문화/레저,영화/공연,11
26,SS029,스포츠/문화/레저,스포츠/문화/레저,스포츠시설,68
27,SS030,스포츠/문화/레저,스포츠/문화/레저,취미/오락,41
28,SS031,스포츠/문화/레저,스포츠/문화/레저,서점,33
29,SS032,스포츠/문화/레저,스포츠/문화/레저용품,스포츠/레저용품,42
30,SS033,스포츠/문화/레저,스포츠/문화/레저용품,문화용품,9
31,SS034,스포츠/문화/레저,스포츠/문화/레저용품,화원,58
32,SS035,여행/교통,숙박,호텔/콘도,75


 
## 새로운 인사이트를 찾아서..

- 사이버 범죄 줄이기? https://www.police.go.kr/www/open/publice/publice0204.jsp
    - https://blog.naver.com/PostView.naver?blogId=gne_education&logNo=221886576742&redirect=Dlog&widgetTypeCall=true&directAccess=false
    - 사이버테러 / 일반사이버 범죄